In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
@everywhere using InvertedIndices
@everywhere using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

In [3]:
gmat = total_symm_calc(positions,dis2mat,f_mat,total_symm_vec)
@benchmark total_symm_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 81 samples with 1 evaluation.
 Range (min … max):  57.448 ms … 66.963 ms  ┊ GC (min … max):  6.87% … 6.12%
 Time  (median):     62.535 ms              ┊ GC (median):    12.61%
 Time  (mean ± σ):   62.162 ms ±  1.578 ms  ┊ GC (mean ± σ):  12.14% ± 1.83%

                                            ▁█▄▄▂              
  ▅▁▃▃▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▅▃▃▃█▁▃▃▆█▅██████▃█▅▁▃▅▁▁▃▁▃ ▁
  57.4 ms         Histogram: frequency by time        64.3 ms <

 Memory estimate: 156.23 MiB, allocs estimate: 2046565.

In [4]:
file = open("/home/ghunter/Documents/RuNNer-master (copy)/Brass_potential/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])
truevals ≈ gmat


false

In [5]:
exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk) = exp(-η*(r2_ij+r2_ik+r2_jk))* f_ij * f_ik * f_jk

theta_part(θ,λ,ζ) = (1+λ*θ)^ζ

function symmfunc_calc(θ_vec,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)

    exp_part = exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)
    g_values = [exp_part* theta_part(θ,λ,ζ) for θ in θ_vec]

    return g_values
end

function ang_symm_val(position1,position2,position3,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)

    θ_vec = all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)
    
    g_vals = symmfunc_calc(θ_vec,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)
    
    return g_vals
end



function symmetry_vals(positions,dist2_mat,f_mat,g_vec,symm_func::AngularType3)
    N = length(g_vec)
    η,λ,ζ = symm_func.eta,symm_func.lambda,symm_func.zeta
    if symm_func.type_vec == [1.,1.,1.]
        for atomindex in eachindex(g_vec)
            for index2 in (atomindex+1):N
                for index3 in (index2+1):N

                    g_val = ang_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                    g_val *= symm_func.tpz 
                    
                    g_vec[atomindex] += g_val[1]
                    g_vec[index2] += g_val[2]
                    g_vec[index3] += g_val[3]

                end
            end
        end
    else 
        g_vec = zeros(N)
    end

    return g_vec
    
end
function symmetry_vals(positions,dist2_mat,f_mat,g_vec,symm_func::RadialType2)
    return calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,symm_func::RadialType2)
end
g_vec = zeros(55)
g_vec = symmetry_vals(positions,dis2mat,f_mat,g_vec,total_symm_vec[11])
g_vec ≈ truevals[11,:]

true

In [6]:
g_vec = zeros(55)
@benchmark calc_symm_vals!($positions,$dis2mat,$f_mat,$g_vec,$total_symm_vec[11])

BenchmarkTools.Trial: 8001 samples with 1 evaluation.
 Range (min … max):  605.541 μs … 768.452 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     622.954 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   624.291 μs ±  12.243 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇▁   ▁▁          █▂▄   ▂           █▃▇                         
  ██▇▇▅██▅▄▄▅▄▃▄▃▃▃████▇▇█▇▆▅▅▄▄▄▄▃▄▄████▇▅▆▄▄▃▄▃▃▃▂▃▂▂▃▃▄▄▂▂▂▁ ▄
  606 μs           Histogram: frequency by time          651 μs <

 Memory estimate: 80 bytes, allocs estimate: 1.

In [7]:
g_vec = zeros(55)
@benchmark symmetry_vals($positions,$dis2mat,$f_mat,$g_vec,$total_symm_vec[11])

BenchmarkTools.Trial: 2241 samples with 1 evaluation.
 Range (min … max):  1.622 ms … 8.368 ms  ┊ GC (min … max):  0.00% … 62.80%
 Time  (median):     1.787 ms             ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.228 ms ± 1.402 ms  ┊ GC (mean ± σ):  14.17% ± 15.89%

  ▇█▆▄▁ ▃▂                                             ▂▁    
  ████████▆▃▃▆█▇▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▅▇▆▃▁▁▁▁▄▇████ █
  1.62 ms     Histogram: log(frequency) by time     7.58 ms <

 Memory estimate: 6.00 MiB, allocs estimate: 78706.

In [8]:
function total_calc(positions,dist2_mat,f_mat,total_symm_vec)
    g_mat = zeros(length(total_symm_vec),size(dist2_mat)[1])
    for g_index in eachindex(total_symm_vec)
        g_mat[g_index,:] = symmetry_vals(positions,dist2_mat,f_mat,g_mat[g_index,:],total_symm_vec[g_index])
    end

    return g_mat

end
@benchmark total_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 84 samples with 1 evaluation.
 Range (min … max):  53.402 ms … 64.820 ms  ┊ GC (min … max):  8.36% … 17.40%
 Time  (median):     60.191 ms              ┊ GC (median):    14.54%
 Time  (mean ± σ):   59.696 ms ±  2.356 ms  ┊ GC (mean ± σ):  13.82% ±  2.39%

         ▁                         █▆▆▆█▆ ▁▃                   
  ▆▄▆▄▁▄▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▆▇██████▆██▇▁▇▁▄▁▁▁▁▄▁▁▁▁▁▁▄ ▁
  53.4 ms         Histogram: frequency by time        64.5 ms <

 Memory estimate: 156.23 MiB, allocs estimate: 2046565.

In [58]:
gtest = total_calc(positions,dis2mat,f_mat,total_symm_vec)

88×55 Matrix{Float64}:
 14.6407       11.9666       11.9666       …  7.45537      5.69954
  0.0           0.0           0.0             0.0          0.0
  7.16978       6.36224       6.36224         3.92888      2.94156
  0.0           0.0           0.0             0.0          0.0
  4.52421       4.14311       4.14311         2.54596      1.91555
  0.0           0.0           0.0          …  0.0          0.0
  0.972533      0.857168      0.857168        0.508399     0.410771
  0.0           0.0           0.0             0.0          0.0
  0.00179489    0.00117603    0.00117603      0.000528536  0.000557398
  0.0           0.0           0.0             0.0          0.0
  ⋮                                        ⋱               
  0.00742548    0.00500205    0.00500205      0.00192165   0.00165975
  0.0           0.0           0.0          …  0.0          0.0
  0.0           0.0           0.0             0.0          0.0
  0.000947854   0.000598351   0.000598351     0.000182036  0.00013

In [59]:
gtest ≈ truevals

true

# Testing the waters with iterating over symmetry functions

NB This method failed. It was significantly slower due to the iterators and memory access.